In [1]:
import sys
import numpy as np
import import_ipynb
from pathlib import Path
from scipy.sparse.linalg import spsolve

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from pre_processing import create_domain, read_mesh
    from fem_processing import boundary_conditions,matrices_assembly
    from pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!


# Project 1: Coaxial Cable

Considere um cabo coaxial com dois dielétricos apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_problem.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

O raio do condutor interno é ``a`` e do condutor externo é ``b``. O primeiro dielétrico, $\varepsilon_{1}$, preenche o anel circular entre os raios ``a`` e ``c``, enquanto o segundo dielétrico, $\varepsilon_{2}$, preenche o anel entre os raios ``c`` e ``b``.  

Entre o condutor interno e o externo é aplicada uma diferença de potencial igual a $V$.  

Seja $V = 1$ (potencial zero em ``a`` e um em ``b``), $a = 2 \, mm$, $b = 8 \, mm$, $c = 5 \, mm$, $\varepsilon_{r1} = 2$ e $\varepsilon_{r2} = 4$. Para este problema, além da convergência nas normas $L_2$ e da energia, calcule a convergência para o valor da capacitância do dispositivo.

# 9-noded Quadratic Quadrilateral Lagrangian Element $Q_2$
## Pre-processor module
### Geometry and mesh Domain

In [2]:
FINITE_ELEMENT = ("Quadrangle", 2)

BOUNDARY = [{'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'inner_conductor'},
                       {'type': 'Dirichlet', 'tag': 102, 'value': 1.0, 'name': 'outer_conductor'}]

MATERIAL = [{'type': 'relative_electric_permittivity', 'tag': 201, 'value': 2, 'name': 'medium_1'},
                        {'type': 'relative_electric_permittivity', 'tag': 202, 'value': 4, 'name': 'medium_2'}]

# Create mesh from file geometry
create_domain.coaxial_geometry(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=0.5E-3, view_mesh=True)
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='coaxial')
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='coaxial')

Model coaxial_domain_Quadrangle2 (2D)
Info     : 8 geometric entities
Info     : 4 Physical Groups
Info     : 3368 nodes in total
Info     : 810 2-D elements in total
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pre_processing\pictures\coaxial_meshed_domain_Quadrangle2.svg


### Mesh Data Dictionaries
#### Node Data

In [3]:
mesh_nodes = mesh_data['nodes_data']; mesh_nodes[:1]

[{'TagID': 1,
  'global_coord': (0.002, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}}]

#### Conn Data

In [4]:
conn_data = mesh_data['conn_data']; conn_data[:1]

[{'TagID': 196,
  'conn_list': [628, 508, 570, 605, 630, 631, 632, 633, 634],
  'material': {'type': 'relative_electric_permittivity',
   'tag': 201,
   'value': 2,
   'name': 'medium_1'}}]

### Figure 1: Coaxial meshed domain 

In [5]:
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='coaxial', Numbering=False)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pre_processing\pictures\coaxial_meshed_domain_Quadrangle2.svg


<figure>
    <img src="pre_processing/pictures/coaxial_meshed_domain_Quadrangle2.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Coaxial meshed domain.</figcaption>
</figure>

In [6]:
nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
conn = [element['conn_list'] for element in mesh_data['conn_data']]

#### `apply_physics`

In [7]:
mesh_data = ps.apply_physics(mesh_data, FINITE_ELEMENT)
mesh_data['conn_data'][:1]

[{'TagID': 196,
  'conn_list': [628, 508, 570, 605, 630, 631, 632, 633, 634],
  'material': {'type': 'relative_electric_permittivity',
   'tag': 201,
   'value': 2,
   'name': 'medium_1',
   'a_value': 1.77083756256e-11},
  'source': {'type': 'free_source', 'value': 0}}]

### Imposition of Boundary Conditions

In [8]:
dirichlet_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
dirichlet_values = [
    node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
free_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Free']

print(f"The entire domain has {len(nodes)} nodes: " 
      f"{len(nodes) - len(dirichlet_nodes)} free nodes and {len(dirichlet_nodes)} Dirichlet nodes.")
print("Free nodes: \n", np.array(free_nodes))
print("Dirichlet nodes: \n", np.array(dirichlet_nodes))
print("Dirichlet nodes values: \n", np.array(dirichlet_values))

The entire domain has 3368 nodes: 3112 free nodes and 256 Dirichlet nodes.
Free nodes: 
 [   2   55   56 ... 3366 3367 3368]
Dirichlet nodes: 
 [  1   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54 182
 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200
 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236
 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254
 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272
 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308
 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326
 327 328 329 330 331 332 333 334 335 336 337 338 339 

## Processor Module
### Reduced Global matrix $A_{gr}$ with boundary conditions

In [9]:
Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, FINITE_ELEMENT)
print("Reduced global matrix shape:", Agr.shape)
print("Reduced global matrix shape: ", Agr.shape)
print("Reduced force vector shape: ", fgr.shape)

Reduced global matrix shape: (3112, 3112)
Reduced global matrix shape:  (3112, 3112)
Reduced force vector shape:  (3112, 1)


### Global Potential Vector

In [10]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.get_global_potentials(mesh_data, ur)

## Post-Processor

In [11]:
graph_results.approximate_not_structured_quadrangles(FINITE_ELEMENT, mesh_data, potential_u, model='coaxial')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pos_processing\pictures\coaxial_domain_solution_Quadrangle2_not_structured.svg


### Figure 2: Scalar Field Distribution

<figure>
    <img src="pos_processing/pictures/coaxial_domain_solution_Quadrangle2_not_structured.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Post-Processor: Scalar Field Distribution.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.